# data

In [1]:
from os import listdir
from os.path import isfile, join
easy_ham = [f for f in listdir("data/easy_ham") if isfile(join("data/easy_ham", f))] # 2500 = 1716+784
easy_ham_2 = [f for f in listdir("data/easy_ham_2") if isfile(join("data/easy_ham_2", f))] # 1400 = 960+440
hard_ham = [f for f in listdir("data/hard_ham") if isfile(join("data/hard_ham", f))] # 250 = 172+78
spam = [f for f in listdir("data/spam") if isfile(join("data/spam", f))] # 500 = 344+156
spam_2 = [f for f in listdir("data/spam_2") if isfile(join("data/spam_2", f))] # 1396 = 958+438
for index in range(len(easy_ham)):
    easy_ham[index] = "data/easy_ham/"+ easy_ham[index]
for index in range(len(easy_ham_2)):
    easy_ham_2[index] = "data/easy_ham_2/"+ easy_ham_2[index]
for index in range(len(hard_ham)):
    hard_ham[index] = "data/hard_ham/"+ hard_ham[index]
for index in range(len(spam)):
    spam[index] = "data/spam/"+ spam[index]
for index in range(len(spam_2)):
    spam_2[index] = "data/spam_2/"+ spam_2[index]

In [2]:
Training_Ham = easy_ham[0:1716]
Training_Ham += easy_ham_2[0:960]
Training_Ham += hard_ham[0:172]
len(Training_Ham)

2848

In [3]:
Testing_Ham = easy_ham[1716+1:]
Testing_Ham += easy_ham_2[960+1:]
Testing_Ham += hard_ham[172+1:]
len(Testing_Ham)

1299

In [4]:
Training_Spam = spam[0:344]
Training_Spam += spam_2[0:958]
len(Training_Spam)

1302

In [5]:
Testing_Spam = spam[344+1:]
Testing_Spam += spam_2[958+1:]
len(Testing_Spam)

592

In [6]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
s=set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [7]:
import re
import codecs
frequency_list = [1, 2, 3, 5, 10, 20, 30, 50, 100, 200, 300, 500, 1000, 2000, 10000]
# frequency_list = [10000]

dictionary = dict()
Training = Training_Ham + Training_Spam
Testing = Testing_Ham + Testing_Spam


In [8]:
import numpy as np
import pandas as pd
from sklearn.svm import SVC
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
f = open("ret/SVM.txt", "a")
for frequency in frequency_list:
    print('frequency is ' + str(frequency))
    f.write('\nfrequency is ' + str(frequency) + '\n')
    dictionary = dict()
    
    for file_names in Training:
        #f = open(file_names, "r")
        ## UnicodeDecodeError: 'cp950' codec can't decode byte 0x93 in position 1885: illegal multibyte sequence
        #f = open(file_names, "r", encoding="utf-8")
        ## UnicodeDecodeError: 'utf-8' codec can't decode byte 0x93 in position 1885: invalid start byte

        file = codecs.open(file_names, 'r', encoding='utf-8', errors='ignore')

        body = file.read().split("\n\n", 1)[1].lower()
        #word = re.findall(r"[\w']+", body) # 93783 ## 21525
        word = re.findall(r"[a-zA-Z']+", body) # 123656 ## 21224
        for i in word:
            dictionary[i] = dictionary.get(i, 0) + 1
#     print(len(dictionary))
    dictionary = {i:dictionary[i] for i in dictionary if dictionary[i]>frequency}
#     print(len(dictionary))
    #print(dictionary)
    list_dictionary = list(dictionary)
    list_dictionary = list(filter(lambda w: not w in s,list_dictionary))
    list_dictionary
#     print(len(list_dictionary))
    
    # Remove common word in dictionary
    for s__ in s:
        if s__ in dictionary:
            del dictionary[s__]
#     print(len(dictionary))

    print('dict length: ' + str(len(dictionary)))
    f.write('dict length: ' + str(len(dictionary)))
    count = 0
    for i in dictionary:
    #     print(i)
        dictionary[i] = count
        count += 1
    dictionary

    # ****************** train *****************
    train_ndarray = np.zeros((len(Training), len(dictionary)), dtype = float)
    train_ndarray.shape

    label_train = [0] * len(Training_Ham) + [1] * len(Training_Spam)
    label_test = [0] * len(Testing_Ham) + [1] * len(Testing_Spam)

    count = 0
    for Ham_mail in Training_Ham:
        file = codecs.open(Ham_mail, 'r', encoding='utf-8', errors='ignore')
        body = file.read().split("\n\n", 1)[1].lower()
        word = re.findall(r"[a-zA-Z']+", body)
#         dict_count = 0
        for w in word:
            if w in dictionary:
                train_ndarray[count][dictionary[w]] = 1
#                 dict_count += 1
#         train_ndarray[count][-1] = dict_count/len(word)
        count += 1
        
    for Spam_mail in Training_Spam:
        file = codecs.open(Spam_mail, 'r', encoding='utf-8', errors='ignore')
        body = file.read().split("\n\n", 1)[1].lower()
        word = re.findall(r"[a-zA-Z']+", body)
#         dict_count = 0
        for w in word:
            if w in dictionary:
                train_ndarray[count][dictionary[w]] = 1
#                 dict_count += 1
#         train_ndarray[count][-1] = dict_count/len(word)
        count += 1
    train_dataframe = pd.DataFrame(train_ndarray)
#     train_dataframe.rename(columns={len(dictionary):'whole_dict_propotion'})
    
# ****************** test ******************
    
    test_ndarray = np.zeros((len(Testing), len(dictionary)), dtype = float)
    test_ndarray.shape

    count = 0
    for Ham_mail in Testing_Ham:
        file = codecs.open(Ham_mail, 'r', encoding='utf-8', errors='ignore')
        body = file.read().split("\n\n", 1)[1].lower()
        word = re.findall(r"[a-zA-Z']+", body)
#         dict_count = 0
        for w in word:
            if w in dictionary:
                test_ndarray[count][dictionary[w]] = 1
#                 dict_count += 1
#         test_ndarray[count][-1] = dict_count/len(word)
    #     print(train_ndarray[count])
        count += 1
    for Spam_mail in Testing_Spam:
        file = codecs.open(Spam_mail, 'r', encoding='utf-8', errors='ignore')
        body = file.read().split("\n\n", 1)[1].lower()
        word = re.findall(r"[a-zA-Z']+", body)
#         dict_count = 0
        for w in word:
            if w in dictionary:
                test_ndarray[count][dictionary[w]] = 1
#                 dict_count += 1
#         test_ndarray[count][-1] = dict_count/len(word)
        count += 1
    test_dataframe = pd.DataFrame(test_ndarray)
#     test_dataframe.rename(columns={len(dictionary):'whole_dict_propotion'})
    
# **************************************************************

    SVM = SVC(kernel='linear')
    SVM.fit(train_dataframe, label_train)

    result = SVM.predict(test_dataframe)

    
    #-------------- from block -----------------
    
#     Spam_from = dict()
#     count = 0
#     for Spam_mail in Training_Spam:
#         file = codecs.open(Spam_mail, 'r', encoding='utf-8', errors='ignore')
#         word = re.findall(r"From.*\n", file.read())[0]
#         word_2 = re.findall(r"[\w*\.?]*@[\w*\.?]*", word)
#         if not word_2 :
#             continue
#         Spam_from[word_2[0]] = Spam_from.get(i, 0)    
    
#     index = 0
#     cheat_result = result.copy()
#     for mail in Testing:
#         file = codecs.open(mail, 'r', encoding='utf-8', errors='ignore')
#         word = re.findall(r"From.*\n", file.read())[0]
#         word_2 = re.findall(r"[\w*\.?]*@[\w*\.?]*", word)
#         if word_2[0] in Spam_from:
#             cheat_result[index] = 1
#         index += 1
    
#     eva_fun_2(label_test, cheat_result, 'NN')
    
    #--------------- end ---------------------
    
     
    tn, fp, fn, tp = confusion_matrix(label_test, result).ravel()

    Accuracy = (tp+tn)/(tn+ fp+ fn+ tp)
    Precision = (tp)/(fp+ tp)
    Recall = (tp)/(fn+ tp)
    print('Recall\t\tPrecision\tAccuracy')
    f.write('\nRecall\t\tPrecision\tAccuracy\n')
    print(str(round(Recall*100, 2))+'\t\t'+ str(round(Precision*100, 2))+'\t\t'+str(round(Accuracy*100, 2)))
    f.write(str(round(Recall*100, 2))+'\t\t'+ str(round(Precision*100, 2))+'\t\t'+str(round(Accuracy*100, 2)))
    print('\n------------------------------------\n')
    f.write('\n------------------------------------\n')
f.close()

frequency is 1
dict length: 43861
Recall		Precision	Accuracy
96.79		94.4		97.2

------------------------------------

frequency is 2
dict length: 26435
Recall		Precision	Accuracy
97.13		94.57		97.36

------------------------------------

frequency is 3
dict length: 21068
Recall		Precision	Accuracy
96.79		94.55		97.25

------------------------------------

frequency is 5
dict length: 15425
Recall		Precision	Accuracy
96.62		94.39		97.14

------------------------------------

frequency is 10
dict length: 9890
Recall		Precision	Accuracy
96.62		94.55		97.2

------------------------------------

frequency is 20
dict length: 6247
Recall		Precision	Accuracy
96.79		94.71		97.3

------------------------------------

frequency is 30
dict length: 4690
Recall		Precision	Accuracy
96.79		94.55		97.25

------------------------------------

frequency is 50
dict length: 3170
Recall		Precision	Accuracy
96.79		94.55		97.25

------------------------------------

frequency is 100
dict length: 1746
Recall		P

In [9]:
import time
 
localtime = time.asctime( time.localtime(time.time()) )
print(localtime)

Fri Jun 12 15:40:55 2020


In [10]:
# Remove common word in dictionary
for s__ in s:
    if s__ in dictionary:
        del dictionary[s__]
print(len(dictionary))

18


In [11]:
count = 0
for i in dictionary:
#     print(i)
    dictionary[i] = count
    count += 1
dictionary

{'com': 0,
 'p': 1,
 'http': 2,
 'www': 3,
 'face': 4,
 'width': 5,
 'b': 6,
 'size': 7,
 'font': 8,
 'nbsp': 9,
 'br': 10,
 'href': 11,
 'gif': 12,
 'src': 13,
 'tr': 14,
 'td': 15,
 'img': 16,
 'height': 17}

In [12]:
import numpy as np
train_ndarray = np.zeros((len(Training), len(dictionary)), dtype = int)
train_ndarray.shape

(4150, 18)

In [13]:
label_train = [0] * len(Training_Ham) + [1] * len(Training_Spam)
label_test = [0] * len(Testing_Ham) + [1] * len(Testing_Spam)

In [14]:
import pandas as pd
count = 0
for Ham_mail in Training_Ham:
    file = codecs.open(Ham_mail, 'r', encoding='utf-8', errors='ignore')
    body = file.read().split("\n\n", 1)[1].lower()
    word = re.findall(r"[a-zA-Z']+", body)
    for w in word:
        if w in dictionary:
            train_ndarray[count][dictionary[w]] = 1
#     print(train_ndarray[count])
    count += 1
for Spam_mail in Training_Spam:
    file = codecs.open(Spam_mail, 'r', encoding='utf-8', errors='ignore')
    body = file.read().split("\n\n", 1)[1].lower()
    word = re.findall(r"[a-zA-Z']+", body)
    for w in word:
        if w in dictionary:
            train_ndarray[count][dictionary[w]] = 1
    count += 1
train_dataframe = pd.DataFrame(train_ndarray)

In [15]:
import numpy as np
test_ndarray = np.zeros((len(Testing), len(dictionary)), dtype = int)
test_ndarray.shape

(1891, 18)

In [16]:
import pandas as pd
count = 0
for Ham_mail in Testing_Ham:
    file = codecs.open(Ham_mail, 'r', encoding='utf-8', errors='ignore')
    body = file.read().split("\n\n", 1)[1].lower()
    word = re.findall(r"[a-zA-Z']+", body)
    for w in word:
        if w in dictionary:
            test_ndarray[count][dictionary[w]] = 1
#     print(train_ndarray[count])
    count += 1
for Spam_mail in Testing_Spam:
    file = codecs.open(Spam_mail, 'r', encoding='utf-8', errors='ignore')
    body = file.read().split("\n\n", 1)[1].lower()
    word = re.findall(r"[a-zA-Z']+", body)
    for w in word:
        if w in dictionary:
            test_ndarray[count][dictionary[w]] = 1
    count += 1
test_dataframe = pd.DataFrame(test_ndarray)

In [17]:
from sklearn.neighbors import KNeighborsClassifier
neigh = KNeighborsClassifier(n_neighbors=5)
neigh.fit(train_dataframe, label_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                     weights='uniform')

In [18]:
result = neigh.predict(test_dataframe)

In [19]:
eva_fun_2(label_test, result, 'K-nn')

NameError: name 'eva_fun_2' is not defined

In [ ]:
count = 0
for Spam_mail in Training_Spam:
    file = codecs.open(Spam_mail, 'r', encoding='utf-8', errors='ignore')
#     print(file.read())
    word = re.findall(r"From.*\n", file.read())[0]
#     print(Spam_mail)
    word_2 = re.findall(r"[\w*\.?]*@[\w*\.?]*", word)
    if not word_2 :
        continue
    Spam_from[word_2[0]] = Spam_from.get(i, 0)
#     print(word_2[0])
    

In [ ]:
Spam_from

In [ ]:
count = 0
cheat_result = result.copy()
for mail in Testing:
    file = codecs.open(mail, 'r', encoding='utf-8', errors='ignore')
#     print(file.read())
    word = re.findall(r"From.*\n", file.read())[0]
#     print(Spam_mail)
    word_2 = re.findall(r"[\w*\.?]*@[\w*\.?]*", word)
#     if word_2[0] == "admin@xent.com":
#         continue
#     if word_2[0] == "admin@lists.sourceforge.net":
#         continue
#     if word_2[0] == "admin@linux.ie":
#         continue
    if word_2[0] in Spam_from:
        cheat_result[count] = 1
        print(word_2[0])
#     print(word_2[0])
    count += 1

In [ ]:
cheat_result

In [ ]:
eva_fun_2(label_test, cheat_result, 'K-nn')

In [ ]:
for i in range(len(cheat_result)):
    if cheat_result[i] != result[i]:
        print(i)